In [ ]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
DATA_DIR = os.path.join("data", "argentina")
JSON_RESULTS = os.path.join(DATA_DIR, "simulation_results.json")

# Generate data

In [ ]:
import generate_databases
generate_databases.store_all()

In [ ]:
!cd simulation && cmake -DCMAKE_BUILD_TYPE=Release . && make -j8

In [ ]:
print("Running simulation...")
!simulation/simulation \
    --days 300 \
    --population {os.path.join(DATA_DIR, "fake_population")} \
    --json {JSON_RESULTS} \
    --seed 0 \
    --silent

# Show results

In [ ]:
with open(JSON_RESULTS) as f:
  sim_res = json.load(f)
sim_res = pd.DataFrame(sim_res)
sim_res.head()

In [ ]:
print(f"Tiempo total de la simulación: {sim_res['compute_time_ms'].sum()/1000/60:.2f} minutos")
fig = go.Figure()
fig.add_trace(go.Scatter(x=sim_res["day"], y=sim_res["compute_time_ms"], name=""))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


fig.update_layout(title='Performance de la simulación',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Duración (milisegundos)')


fig.show()

In [ ]:
states = {
    "SUSCEPTIBLE": "Individuos susceptibles",
    "EXPOSED": "Individuos expuestos",
    "INFECTED_1": "Individuos infectados",
    "INFECTED_2": "Individuos infectados severamente",
    "INFECTED_3": "Individuos infectados críticamente",
    "RECOVERED": "Individuos recuperados",
    "DEAD": "Individuos muertos"
}

In [ ]:
poblacion_total = sim_res[sim_res['day']==1][states].sum(axis=1)[0]
f"{poblacion_total} personas"

In [ ]:
fig = go.Figure()
for st,name in states.items():
    fig.add_trace(go.Scatter(x=sim_res["day"], y=sim_res[st]*1000/poblacion_total, name=name))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Casos pronosticados de COVID-19 por resultado clínico',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Casos por cada mil personas')


fig.show()

In [ ]:
inf_sources = {
    "HOME_CONTACT": "Contacto en hogar",
    "SCHOOL_CONTACT": "Contacto en la escuela",
    "WORK_CONTACT": "Contacto en trabajo",
    "NEIGHBOURHOOD_CONTACT": "Contacto en vecindario",
    "INTER_NEIGHBOURHOOD_CONTACT": "Contacto en vecindario cercano",
    "IMPORTED_CASE": "Contacto fuera del país",
}

In [ ]:
fig = go.Figure(data=[go.Pie(labels=list(inf_sources.values()), values=sim_res[inf_sources].sum())])
fig.update_layout(title="Fuentes de infecciones")
fig.show()

In [ ]:
fig = go.Figure()
for st,name in inf_sources.items():
    fig.add_trace(go.Scatter(x=sim_res["day"], y=sim_res[st]*1000/poblacion_total, name=name))
                         #line=dict(color='firebrick', width=4)))# dash options include 'dash', 'dot', and 'dashdot'


# Edit the layout
fig.update_layout(title='Fuente de infecciones',
                   xaxis_title='Tiempo desde el primer caso (Días)',
                   yaxis_title='Infecciones por cada mil personas')


fig.show()